In [5]:
import os
import glob
from pathlib import Path
from typing import Optional, List

import torch
import pandas as pd
# from .sae.inference import load_sae
import sys
# if '/maiziezhou_lab2/yunfei/Projects/interpTFM/sae' not in sys.path:
sys.path.insert(0, '/maiziezhou_lab2/yunfei/Projects/interpTFM')
from sae.inference import load_sae

# ------------------------------
# Config
# ------------------------------
GENE_IDS_ROOT = "/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/gene_ids"
ACTS_ROOT     = "/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4"
OUT_FILENAME  = "sae_latents.parquet"     # per-shard output
ACTS_FILENAME = "activations.pt"          # per-shard input
META_FILENAME = "cell_gene_pairs.txt"     # per-shard metadata (cell_id, gene_id)

# SAE checkpoint
CKPT_PATH     = "/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/sae_latents/sae_output_layer4/ae_normalized.pt"

# Encoding options
BATCH_SIZE    = 4096
CLS_ONLY      = False      # True => encode only <cls> token rows per shard
SKIP_EXISTING = True       # True => skip shard if OUT_FILENAME exists


# ------------------------------
# Helpers
# ------------------------------
def find_shards(root: str) -> List[Path]:
    return sorted([Path(p) for p in glob.glob(os.path.join(root, "shard_*")) if os.path.isdir(p)])

def read_meta(path: Path) -> pd.DataFrame:
    # Assumes 2 columns: cell_id, gene_id (TSV or whitespace)
    try:
        df = pd.read_csv(path, sep="\t", header=None, names=["cell_id", "gene_id"])
    except Exception:
        df = pd.read_csv(path, sep=r"\s+", header=None, names=["cell_id", "gene_id"])
    return df

def pick_cls_indices(meta_df: pd.DataFrame) -> torch.Tensor:
    # accept multiple possible <cls> spellings
    cls_mask = meta_df["gene_id"].astype(str).isin({"<cls>", "<CLS>", "CLS", "cls"})
    return torch.tensor(cls_mask.values.nonzero()[0], dtype=torch.long)

@torch.no_grad()
def sae_encode_batched(X: torch.Tensor, model: torch.nn.Module, batch_size: int, device: torch.device) -> torch.Tensor:
    outs = []
    for i in range(0, X.shape[0], batch_size):
        xb = X[i:i+batch_size].to(device, non_blocking=True)
        # Try common SAE interfaces
        if hasattr(model, "encode"):
            z = model.encode(xb)
        elif hasattr(model, "encoder"):
            z = model.encoder(xb)
        else:
            out = model(xb)
            if isinstance(out, (tuple, list)) and len(out) >= 2:
                # heuristically pick the non-input-dim output as latents
                z = out[0] if out[0].ndim == 2 and out[0].shape[1] != xb.shape[1] else out[1]
            else:
                raise RuntimeError("SAE object lacks encode/encoder; customize sae_encode_batched for your model.")
        outs.append(z.detach().cpu())
    return torch.cat(outs, dim=0)


# ------------------------------
# Main
# ------------------------------
def encode_all_shards(
    gene_ids_root: str = GENE_IDS_ROOT,
    acts_root: str = ACTS_ROOT,
    ckpt_path: str = CKPT_PATH,
    acts_filename: str = ACTS_FILENAME,
    meta_filename: str = META_FILENAME,
    out_filename: str = OUT_FILENAME,
    batch_size: int = BATCH_SIZE,
    cls_only: bool = CLS_ONLY,
    skip_existing: bool = SKIP_EXISTING,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[INFO] loading SAE from {ckpt_path} on {device} ...")
    sae = load_sae(ckpt_path).to(device).eval()

    gid_shards = find_shards(gene_ids_root)
    act_shards = {p.name: p for p in find_shards(acts_root)}
    print(f"[INFO] found {len(gid_shards)} gene_id shards; {len(act_shards)} activation shards.")

    for gid_shard in gid_shards:
        shard_id = gid_shard.name
        acts_dir = act_shards.get(shard_id)
        if acts_dir is None:
            print(f"[WARN] no activations dir for {shard_id}; skipping.")
            continue

        out_path = acts_dir / out_filename
        if skip_existing and out_path.exists():
            print(f"[SKIP] {shard_id}: {out_filename} already exists.")
            continue

        meta_path = gid_shard / meta_filename
        acts_path = acts_dir / acts_filename
        if not meta_path.exists():
            print(f"[WARN] {shard_id}: missing {meta_filename}; skipping.")
            continue
        if not acts_path.exists():
            print(f"[WARN] {shard_id}: missing {acts_filename}; skipping.")
            continue

        print(f"[INFO] {shard_id}: loading meta + activations ...")
        meta_df = read_meta(meta_path)
        X = torch.load(str(acts_path))  # [N_tokens, H]
        if not isinstance(X, torch.Tensor):
            raise TypeError(f"{acts_path} did not load as a torch.Tensor.")
        if meta_df.shape[0] != X.shape[0]:
            raise AssertionError(f"{shard_id}: meta rows ({meta_df.shape[0]}) != activations rows ({X.shape[0]})")

        if cls_only:
            cls_idx = pick_cls_indices(meta_df)
            if cls_idx.numel() == 0:
                print(f"[WARN] {shard_id}: no <cls> tokens; skipping.")
                continue
            # Subselect both meta and activations to <cls> rows
            meta_sub = meta_df.iloc[cls_idx.numpy()].reset_index(drop=True)
            X_sub = X.index_select(0, cls_idx)
        else:
            meta_sub = meta_df.reset_index(drop=True)
            X_sub = X

        print(f"[INFO] {shard_id}: encoding {X_sub.shape[0]} tokens to SAE latents ...")
        Z = sae_encode_batched(X_sub, sae, batch_size=batch_size, device=device)  # [N_rows, K_latents]
        n_latents = Z.shape[1]

        # Build a DataFrame with integer latent columns
        df_out = pd.DataFrame(Z.numpy(), index=meta_sub.index, columns=list(range(n_latents)))
        # Carry along identifiers (nice for debugging / downstream joins)
        df_out.insert(0, "cell_id", meta_sub["cell_id"].values)
        df_out.insert(1, "gene_id", meta_sub["gene_id"].values)

        out_path.parent.mkdir(parents=True, exist_ok=True)
        df_out.to_parquet(out_path)
        print(f"[OK]  {shard_id}: wrote {out_path}  (shape={df_out.shape})")

        # Free tensors early
        del X, X_sub, Z, df_out, meta_df

    print("[DONE] all shards processed.")


if __name__ == "__main__":
    encode_all_shards()


[INFO] loading SAE from /maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/sae_latents/sae_output_layer4/ae_normalized.pt on cuda ...


/home/huy21/anaconda3/envs/scgpt/lib/python3.8/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


[INFO] found 60 gene_id shards; 60 activation shards.
[INFO] shard_0: loading meta + activations ...
[INFO] shard_0: encoding 516880 tokens to SAE latents ...


/home/huy21/anaconda3/envs/scgpt/lib/python3.8/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


[OK]  shard_0: wrote /maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4/shard_0/sae_latents.parquet  (shape=(516880, 4098))
[INFO] shard_1: loading meta + activations ...
[INFO] shard_1: encoding 509497 tokens to SAE latents ...


...

[OK]  shard_7: wrote /maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4/shard_7/sae_latents.parquet  (shape=(553151, 4098))
[INFO] shard_8: loading meta + activations ...
[INFO] shard_8: encoding 491853 tokens to SAE latents ...


/home/huy21/anaconda3/envs/scgpt/lib/python3.8/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


[OK]  shard_8: wrote /maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4/shard_8/sae_latents.parquet  (shape=(491853, 4098))
[INFO] shard_9: loading meta + activations ...
[INFO] shard_9: encoding 492140 tokens to SAE latents ...


/home/huy21/anaconda3/envs/scgpt/lib/python3.8/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


[OK]  shard_9: wrote /maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4/shard_9/sae_latents.parquet  (shape=(492140, 4098))
[DONE] all shards processed.


In [4]:
sys.path

['/maiziezhou_lab2/yunfei/Projects/interpTFM/sae',
 '/home/huy21/anaconda3/envs/scgpt/lib/python38.zip',
 '/home/huy21/anaconda3/envs/scgpt/lib/python3.8',
 '/home/huy21/anaconda3/envs/scgpt/lib/python3.8/lib-dynload',
 '',
 '/home/huy21/.local/lib/python3.8/site-packages',
 '/home/huy21/anaconda3/envs/scgpt/lib/python3.8/site-packages']